In [2]:
""" LSTM Network.
A RNN Network (LSTM) implementation example using Keras.
This example is using the MNIST handwritten digits dataset (http://yann.lecun.com/exdb/mnist/)

Ressouces:
    [Long Short Term Memory](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf)
    [Understanding LSTMs](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
    [MNIST Dataset](http://yann.lecun.com/exdb/mnist/)

Repository: https://github.com/ar-ms/lstm-mnist
"""

# Imports
import sys

from tensorflow.examples.tutorials.mnist import input_data
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.models import load_model
import numpy as np

In [3]:
time_steps = 28
n_units = 128
n_inputs = 28
n_classes = 10
batch_size = 128
n_epochs=5
_data_loaded = False
_trained = False

In [4]:
model = Sequential()
model.add(LSTM(n_units, input_shape=(time_steps, n_inputs)))
model.add(Dense(n_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
         optimizer='rmsprop',
         metrics=['accuracy'])

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               80384     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 81,674
Trainable params: 81,674
Non-trainable params: 0
_________________________________________________________________


In [6]:
mnist = input_data.read_data_sets("mnist", one_hot=True)
_data_loaded = True

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [7]:
x_train = [x.reshape((-1, time_steps, n_inputs)) for x in mnist.train.images]
x_train = np.array(x_train).reshape((-1, time_steps, n_inputs))

In [8]:
model.fit(x_train, mnist.train.labels,
          batch_size=batch_size, epochs=n_epochs, shuffle=False)

_trained = True

model.save("./saved_model/lstm-model.h5")

Epoch 1/5
55000/55000 [==============================] - 15s 277us/step - loss: 0.6347 - acc: 0.7916
Epoch 2/5
55000/55000 [==============================] - 15s 277us/step - loss: 0.2025 - acc: 0.9376
Epoch 3/5
55000/55000 [==============================] - 17s 301us/step - loss: 0.1235 - acc: 0.9619
Epoch 4/5
55000/55000 [==============================] - 16s 299us/step - loss: 0.0889 - acc: 0.9728
Epoch 5/5
55000/55000 [==============================] - 16s 298us/step - loss: 0.0684 - acc: 0.9790


# KERAS - GAN

In [105]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

### init 1

In [106]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

optimizer = Adam(0.0002, 0.5)

In [107]:
def build_generator():

    noise_shape = (100,)

    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)

    return Model(noise, img)

In [108]:
def build_discriminator():

    img_shape = (img_rows, img_cols, channels)

    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

### init 2

In [109]:
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

optimizer = Adam(0.0002, 0.5)

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

# Build and compile the generator
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

# The generator takes noise as input and generates imgs
z = Input(shape=(100,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
valid = discriminator(img)

# The combined model  (stacked generator and discriminator) takes
# noise as input => generates images => determines validity
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

### treino fora do loop

In [110]:
half_batch = int(batch_size / 2)

In [111]:
noise = np.random.normal(0, 1, (half_batch, 100))

In [112]:
noise.shape

(64, 100)

In [113]:
gen_imgs = generator.predict(noise)

In [114]:
gen_imgs.shape

(64, 28, 28, 1)

In [115]:
(X_train, _), (_, _) = mnist.load_data()

In [116]:
# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

In [117]:
# Select a random half batch of images
idx = np.random.randint(0, X_train.shape[0], half_batch)
imgs = X_train[idx]

In [118]:
imgs.shape

(64, 28, 28, 1)

In [119]:
d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))

/home/eurismar/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [120]:
d_loss_real

[1.4602354, 0.0]

### treino normal dentro do loop

In [121]:
def train(epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    half_batch = int(batch_size / 2)

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]

        noise = np.random.normal(0, 1, (half_batch, 100))

        # Generate a half batch of new images
        gen_imgs = generator.predict(noise)
        print("shape......{}".format(gen_imgs.shape))
        # Train the discriminator
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


        # ---------------------
        #  Train Generator
        # ---------------------

        noise = np.random.normal(0, 1, (batch_size, 100))

        # The generator wants the discriminator to label the generated samples
        # as valid (ones)
        valid_y = np.array([1] * batch_size)

        # Train the generator
        g_loss = combined.train_on_batch(noise, valid_y)

        # Plot the progress
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0:
            sample_images(epoch)

In [122]:
def sample_images(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5
    print('shape:>>>>>{}'.format(gen_imgs.shape))
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [124]:
train(epochs=30, batch_size=32, sample_interval=200)

shape......(16, 28, 28, 1)


/home/eurismar/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.562571, acc.: 78.12%] [G loss: 0.688034]
shape:>>>>>(25, 28, 28, 1)
shape......(16, 28, 28, 1)
1 [D loss: 0.344570, acc.: 87.50%] [G loss: 0.756540]
shape......(16, 28, 28, 1)
2 [D loss: 0.333906, acc.: 84.38%] [G loss: 0.751981]
shape......(16, 28, 28, 1)
3 [D loss: 0.317060, acc.: 90.62%] [G loss: 0.817291]
shape......(16, 28, 28, 1)
4 [D loss: 0.289904, acc.: 93.75%] [G loss: 0.876761]
shape......(16, 28, 28, 1)
5 [D loss: 0.259601, acc.: 100.00%] [G loss: 0.989168]
shape......(16, 28, 28, 1)
6 [D loss: 0.223100, acc.: 100.00%] [G loss: 1.036528]
shape......(16, 28, 28, 1)
7 [D loss: 0.184109, acc.: 100.00%] [G loss: 1.179163]
shape......(16, 28, 28, 1)
8 [D loss: 0.210491, acc.: 96.88%] [G loss: 1.276659]
shape......(16, 28, 28, 1)
9 [D loss: 0.182622, acc.: 96.88%] [G loss: 1.414727]
shape......(16, 28, 28, 1)
10 [D loss: 0.152864, acc.: 100.00%] [G loss: 1.600769]
shape......(16, 28, 28, 1)
11 [D loss: 0.126359, acc.: 100.00%] [G loss: 1.678313]
shape......(16, 28, 2

# GAN - LSTM

In [ ]:
model = Sequential()
model.add(LSTM(n_units, input_shape=(time_steps, n_inputs)))
model.add(Dense(n_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
         optimizer='rmsprop',
         metrics=['accuracy'])

In [ ]:
time_steps, n_inputs

In [ ]:
def build_generator():

    noise_shape = (100,)

    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)

    return Model(noise, img)

In [29]:
def build_generator_LSTM():

    noise_shape = (100,)

    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape((time_steps, n_inputs)))
    model.add(LSTM(28*28))
    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)


    return Model(noise, img)

In [30]:
# Build and compile the generator
generator_LSTM = build_generator_LSTM()
generator_LSTM.compile(loss='binary_crossentropy', optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 256)               1024      
_________________________________________________________________
dense_10 (Dense)             (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 512)               2048      
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              525312    
__________

In [39]:
def build_discriminator_LSTM():
    """
    img_shape = (28,28)

    model = Sequential()
    model.add(LSTM(n_units, input_shape=(time_steps, n_inputs)))
    model.add(Dense(1, activation='softmax'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)
    """

    img_shape = (img_rows, img_cols, channels)

    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [40]:
# Build and compile the discriminator
discriminator_LSTM = build_discriminator_LSTM()
discriminator_LSTM.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


In [41]:
# The generator takes noise as input and generates imgs
z = Input(shape=(100,))
img = generator(z)

In [42]:
img = generator_LSTM(z)

In [43]:
# For the combined model we will only train the generator
discriminator.trainable = False

In [47]:
# The discriminator takes generated images as input and determines validity
valid = discriminator_LSTM(img)

In [49]:
# The combined model  (stacked generator and discriminator) takes
# noise as input => generates images => determines validity


combined_LSTM = Model(z, valid)
combined_LSTM.compile(loss='binary_crossentropy', optimizer=optimizer)

In [164]:
def sample_images_LSTM(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator_LSTM.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5
    gen_imgs = gen_imgs.reshape((gen_imgs.shape[0],28,28,1))
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [165]:
def train_GAN_LSTM(epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
    



    half_batch = int(batch_size / 2)

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]
        #imgs = imgs.reshape(half_batch,28*28)
        noise = np.random.normal(0, 1, (half_batch, 100))

        # Generate a half batch of new images
        gen_imgs = generator_LSTM.predict(noise)
        gen_imgs = gen_imgs.reshape(half_batch,28,28,1)

        # Train the discriminator
        d_loss_real = discriminator_LSTM.train_on_batch(imgs, np.ones((half_batch, 1)))
        
        
        
        
        
        d_loss_fake = discriminator_LSTM.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        

        # ---------------------
        #  Train Generator
        # ---------------------

        noise = np.random.normal(0, 1, (batch_size, 100))

        # The generator wants the discriminator to label the generated samples
        # as valid (ones)
        valid_y = np.array([1] * batch_size)

        # Train the generator
        g_loss = combined_LSTM.train_on_batch(noise, valid_y)

        # Plot the progress
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0:
            sample_images_LSTM(epoch)
        

In [ ]:
train_GAN_LSTM(epochs=3001, batch_size=32, sample_interval=100)

0 [D loss: 2.026002, acc.: 50.00%] [G loss: 0.049882]
1 [D loss: 1.987558, acc.: 50.00%] [G loss: 0.053321]
2 [D loss: 1.968331, acc.: 50.00%] [G loss: 0.056510]
3 [D loss: 1.997583, acc.: 50.00%] [G loss: 0.052400]
4 [D loss: 2.003509, acc.: 50.00%] [G loss: 0.050562]
5 [D loss: 1.974663, acc.: 50.00%] [G loss: 0.052353]
6 [D loss: 1.973714, acc.: 50.00%] [G loss: 0.053708]
7 [D loss: 2.023912, acc.: 50.00%] [G loss: 0.053294]
8 [D loss: 1.984261, acc.: 50.00%] [G loss: 0.052726]


# TENSORFLOW

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
hello = tf.constant("Hello")

In [ ]:
world = tf.constant("World")

In [ ]:
type(hello)

In [ ]:
with tf.Session() as sess:
    result = sess.run(hello+world)

In [ ]:
print(result)

In [ ]:
a = tf.constant(10)

In [ ]:
b = tf.constant(20)

In [ ]:
a + b

In [ ]:
a + b

In [ ]:
with tf.Session() as sess:
    result = sess.run(a+b)

In [ ]:
result

In [ ]:
fill_mat = tf.fill((4,4),10)

In [ ]:
with tf.Session() as sess:
    result = sess.run(fill_mat)

In [ ]:
result

In [ ]:
my_randn = tf.random_normal((4,4),mean=0.0,stddev=1.0)

In [ ]:
with tf.Session() as sess:
    result = sess.run(my_randn)

In [ ]:
result

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
a = sess.run(my_randn)

In [ ]:
my_randn.eval()

In [ ]:
b = tf.constant([1,2])

In [ ]:
b.eval()

In [ ]:
c = tf.constant([[1,2],[3,4]])

In [ ]:
c.eval()

In [ ]:
a = tf.constant(1)

In [ ]:
b = tf.constant(2)

In [ ]:
c = tf.add(a,b)

In [ ]:
c.eval()

In [ ]:
sess.run(c)

In [ ]:
np.prod(img_shape)

In [ ]:
img_shape

In [ ]:
np.prod((10,20,3))

In [ ]:
model.add(Dense(np.prod(img_shape), activation='tanh'))